# knowledge generation
논리지 생성 단계

# 1. 필요 라이브러리 설치
- Newspaper3k와 Selenium을 사용하여 웹 스크래핑을 수행합니다.
- Chromium-browser와 ChromeDriver는 Selenium이 웹 컨텐츠를 효과적으로 스크랩할 수 있도록 합니다.

In [1]:
!pip install --upgrade newspaper3k selenium
!pip install --upgrade chromedriver-autoinstaller
!sudo apt-get update
!sudo apt-get install chromium-browser


Hit:1 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-browser is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 44 not upgraded.


# 2. Python 스크립트
## 1) 라이브러리 불러오기
- 웹 스크래핑을 위한 필수 라이브러리를 불러옵니다.
- newspaper는 복잡한 웹페이지에서 콘텐츠를 추출하고, BeautifulSoup와 requests는 HTML 페이지를 파싱합니다.

In [2]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


## 2) 뉴스 기사 URL 검색
네이버 뉴스를 통해 주어진 검색어로 최신 뉴스 기사 URL을 검색하고 추출합니다.

In [3]:
def get_news_urls(query):
    search_url = f"https://search.naver.com/search.naver?where=news&query={query}&sort=1"
    response = requests.get(search_url, headers={'User-agent': 'Mozilla/5.0'})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        news_urls = [element['href'] for element in soup.find_all('a', class_='news_tit')]
        return news_urls
    else:
        print("Failed to fetch news URLs")
        return []


## 3)뉴스 기사 스크래핑
추출한 URL에서 뉴스 기사의 세부 정보를 추출합니다.

In [4]:
def scrape_news_articles(news_urls):
    articles_data = []
    for url in news_urls:
        try:
            article = Article(url)
            article.download()
            article.parse()
            articles_data.append({
                'url': url,
                'title': article.title,
                'authors': article.authors,
                'publish_date': article.publish_date,
                'text': article.text
            })
        except Exception as e:
            print(f"Failed to scrape article from {url}: {e}")
    return articles_data


## 4) 데이터 정리 및 Selenium 사용
- 서울 지하철 파업/연착/지연 검색
- DataFrame을 생성하고, 텍스트가 없거나 충분하지 않은 경우 Selenium으로 추가 데이터를 수집합니다.
- null 값이거나 텍스트 길이가 100자 이하 일때, 크롤링이 제대로 되지 않은 걸로 간주, selenium을 통해 크롤링

In [5]:
if __name__ == "__main__":
    queries = ["서울 지하철 파업", "서울 지하철 연착", "서울 지하철 지연"]
    all_articles_data = []
    for query in queries:
        news_urls = get_news_urls(query)
        articles_data = scrape_news_articles(news_urls)
        for article_data in articles_data:
            article_data['category'] = query.split(" ")[-1]
        all_articles_data.extend(articles_data)
    df = pd.DataFrame(all_articles_data)
    df = df.replace('', np.nan)

    # Selenium 설정 및 데이터 추가 수집
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--remote-debugging-port=9222')  # This can help avoid some common errors
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    for index, row in df.iterrows():
        if pd.isna(row['text']) or len(row['text']) < 200:
            driver.get(row['url'])
            try:
                fetched_text = driver.find_element(By.TAG_NAME, 'article').text
                df.at[index, 'text'] = fetched_text
            except Exception as e:
                print(f"Error fetching article from {row['url']}: {e}")
    driver.quit()


# 3. 데이터 확인

In [8]:
df.shape

(30, 6)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           30 non-null     object
 1   title         30 non-null     object
 2   authors       30 non-null     object
 3   publish_date  24 non-null     object
 4   text          30 non-null     object
 5   category      30 non-null     object
dtypes: object(6)
memory usage: 1.5+ KB


In [10]:
df.head(30)

,url,title,authors,publish_date,text,category
0,https://www.yna.co.kr/view/AKR2024052306750000...,"서울교통공사 제3노조 ""타임오프 악용 노조간부 처벌·감사해야""",[],2024-05-23 11:36:04+09:00,"최윤선 기자\n\n'무단결근' 노조 간부 7명, 재심사 거쳐 전원 해임 처분\n\n...",파업
1,http://www.labortoday.co.kr/news/articleView.h...,근로시간면제 집단해고 신종 노조탄압인가,[],2024-05-21 07:30:03+09:00,권오훈 공인노무사(공공운수노조 부위원장)\n\n서울교통공사에서 노조 간부 36명에 ...,파업
2,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,[데스크칼럼] 지하철 개통 50주년,[],None,"[데스크칼럼] 지하철 개통 50주년\n\n스무 살, 난생 처음 지하철이라는 걸 타 ...",파업
3,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “먹거리에 공공요금까지”… 물가 도미노 현실화,[],2024-05-02 12:00:00+09:00,누르던 먹거리 물가 터졌다… 외식업계 가격 줄인상\n\n정부 공공요금 동결…가스공사...,파업
4,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “물가 자극하는 대내외 악재 산적”…공공요금 하반기 줄인상 가능성↑,[],2024-05-02 14:39:44+09:00,물가 ‘복병’ 공공요금…도시가스·전기료 동결\n\n누적 적자 한계로 하반기 인상 가...,파업
5,https://biz.newdaily.co.kr/site/data/html/2024...,김밥부터 치킨·피자에 공공요금 인상까지… 가정의 달 물가 융단 폭격,[],2024-04-28 00:00:00,▲ 대형마트 전경. ⓒ뉴데일리DB\n\n김밥·치킨·커피 등 외식품목 가격과 가스·지...,파업
6,https://www.newsis.com/view/?id=NISX20240426_0...,가스·지하철 등 공공요금 도미노 인상 예고[끝모를 물가 충격②],[],2024-04-29 15:09:44+09:00,가스·전기료 5·6월 발표…재무 개선 압박\n3월 파업에 버스 요금 인상 가능성 커...,파업
7,http://weekly.chosun.com/news/articleView.html...,사모펀드에 포획된 버스 준공영제,[],2024-04-26 15:00:00+09:00,서울 시내버스가 12년 만에 총파업에 돌입한 지난 3월 28일 서울 소재 시내버스 ...,파업
8,http://www.kyeongin.com/main/view.php?key=2024...,"김교흥 '제2 경의선숲길', 이용우 '서울 통근 교통혁명', 모경종 '5호선 연장 ...",[],None,[공약으로 보는 22대 국회·(4·끝)] 인천 서구갑·을·병\n\n\n\n\n\n김...,파업
9,https://www.mediapen.com/news/view/912145,1호선 의왕~당정역 구간서 사망사고…상행선 운행 차질,[],None,전동열차 선로 통행 불가…화서·당정역 무정차 통과\n\n[미디어펜=김준희 기자]수도...,파업


# 4. 데이터 전처리
- publish_date의 null 값 처리를 위해 한번더 selenium 진행

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from dateutil import parser
import re

# Setup Chrome WebDriver with headless options and other configurations
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('blink-settings=imagesEnabled=false')  # 이미지 비활성화

# Setup service with ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.set_page_load_timeout(120)  # 타임아웃을 120초로 설정

# Assume 'df' is your DataFrame and it has columns 'url' and 'publish_date'
date_classes = ['news-date', 'date-repoter', 'article_info', 'article_byline', 'dates','dateFont','article_date']

for index, row in df.iterrows():
    if pd.isna(row['publish_date']):
        url = row['url']
        try:
            driver.get(url)
        except TimeoutException:
            print(f"Timeout while loading {url}")
            continue  # 다음 URL로 건너뛰기
        
        publish_date = None
        for date_class in date_classes:
            date_elements = driver.find_elements(By.CLASS_NAME, date_class)
            for date_element in date_elements:
                date_text = date_element.text
                numbers = re.findall(r'\d+', date_text)
                if len(numbers) >= 3:  # YYYY, MM, DD가 필요
                    publish_date = f"{numbers[0]}-{numbers[1].zfill(2)}-{numbers[2].zfill(2)}"
                    break
            if publish_date:
                break
        
        if publish_date:
            try:
                parsed_date = parser.parse(publish_date)
                df.at[index, 'publish_date'] = parsed_date
            except ValueError as e:
                print(f"Error parsing date from {url}: {e}")

# Close the browser
driver.quit()


In [23]:
df.head(30)

,url,title,authors,publish_date,text,category
0,https://www.yna.co.kr/view/AKR2024052306750000...,"서울교통공사 제3노조 ""타임오프 악용 노조간부 처벌·감사해야""",[],2024-05-23 11:36:04+09:00,"최윤선 기자\n\n'무단결근' 노조 간부 7명, 재심사 거쳐 전원 해임 처분\n\n...",파업
1,http://www.labortoday.co.kr/news/articleView.h...,근로시간면제 집단해고 신종 노조탄압인가,[],2024-05-21 07:30:03+09:00,권오훈 공인노무사(공공운수노조 부위원장)\n\n서울교통공사에서 노조 간부 36명에 ...,파업
2,https://www.dnews.co.kr/uhtml/view.jsp?idxno=2...,[데스크칼럼] 지하철 개통 50주년,[],2024-05-21 00:00:00,"[데스크칼럼] 지하철 개통 50주년\n\n스무 살, 난생 처음 지하철이라는 걸 타 ...",파업
3,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “먹거리에 공공요금까지”… 물가 도미노 현실화,[],2024-05-02 12:00:00+09:00,누르던 먹거리 물가 터졌다… 외식업계 가격 줄인상\n\n정부 공공요금 동결…가스공사...,파업
4,http://www.m-i.kr/news/articleView.html?idxno=...,[기획] “물가 자극하는 대내외 악재 산적”…공공요금 하반기 줄인상 가능성↑,[],2024-05-02 14:39:44+09:00,물가 ‘복병’ 공공요금…도시가스·전기료 동결\n\n누적 적자 한계로 하반기 인상 가...,파업
5,https://biz.newdaily.co.kr/site/data/html/2024...,김밥부터 치킨·피자에 공공요금 인상까지… 가정의 달 물가 융단 폭격,[],2024-04-28 00:00:00,▲ 대형마트 전경. ⓒ뉴데일리DB\n\n김밥·치킨·커피 등 외식품목 가격과 가스·지...,파업
6,https://www.newsis.com/view/?id=NISX20240426_0...,가스·지하철 등 공공요금 도미노 인상 예고[끝모를 물가 충격②],[],2024-04-29 15:09:44+09:00,가스·전기료 5·6월 발표…재무 개선 압박\n3월 파업에 버스 요금 인상 가능성 커...,파업
7,http://weekly.chosun.com/news/articleView.html...,사모펀드에 포획된 버스 준공영제,[],2024-04-26 15:00:00+09:00,서울 시내버스가 12년 만에 총파업에 돌입한 지난 3월 28일 서울 소재 시내버스 ...,파업
8,http://www.kyeongin.com/main/view.php?key=2024...,"김교흥 '제2 경의선숲길', 이용우 '서울 통근 교통혁명', 모경종 '5호선 연장 ...",[],2024-04-25 00:00:00,[공약으로 보는 22대 국회·(4·끝)] 인천 서구갑·을·병\n\n\n\n\n\n김...,파업
9,https://www.mediapen.com/news/view/912145,1호선 의왕~당정역 구간서 사망사고…상행선 운행 차질,[],2024-04-15 00:00:00,전동열차 선로 통행 불가…화서·당정역 무정차 통과\n\n[미디어펜=김준희 기자]수도...,파업


In [24]:
df.to_csv('knowledge_generation_data.csv', index=False)